# Training with CFG representation

In [ ]:
!python -m pip list

In [ ]:
!nvidia-smi

In [ ]:
import gc
import os
import sys
import yaml
import glob
import numpy as np
import pickle
import tensorflow as tf
import random
import tqdm
import seaborn as sn
import numpy as np

from collections import defaultdict

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from spektral.datasets import delaunay
from spektral.layers import *
from spektral.utils.convolution import localpooling_filter
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tqdm.contrib.concurrent import process_map, thread_map

from utils import yaml_load, get_section, GraphDataGenerator

%matplotlib inline

In [ ]:
# Defining some parameters for training
N = 300                         # Number of nodes in the graphs
F = 67                          # Original feature dimensionality
n_classes = 4                   # Number of classes
epochs = 200                    # Number of training epochs
learning_rate = 1e-2            # Learning rate
batch_size = 16                 # batch_size
train_size = 0.85
validation_size = 0.05
test_size = 0.10

data_root = './data/cfgs/'
labels_file = './data/cfgs/labels.yaml'
labels = yaml_load(labels_file)
filenames = list(labels.keys())
random.shuffle(filenames)

train_files = filenames[0 : 0+int(len(filenames)*train_size)]
validation_files = filenames[int(len(filenames)*train_size) : int(len(filenames)*train_size)+int(len(filenames)*validation_size)]
test_files = filenames[int(len(filenames)*validation_size) : int(len(filenames)*validation_size)+int(len(filenames)*test_size)]

train_generator = GraphDataGenerator(data_root, train_files, labels, batch_size=batch_size)
validation_generator = GraphDataGenerator(data_root, validation_files, labels, batch_size=batch_size)
test_generator = GraphDataGenerator(data_root, test_files, labels, batch_size=batch_size)

print(f"""Samples:
Total samples: {len(filenames)}
Train samples: {len(train_files)}
Validation samples: {len(validation_files)}
Test samples: {len(test_files)}
""")

In [ ]:
# Model definition
conv   = GraphConv(45,activation='relu',use_bias=False)
mincut = MinCutPool(N // 2)
conv2  = GraphConv(55,activation='relu',use_bias=False)
pool   = GlobalAttnSumPool()

# First Graph Layers
X1_in = Input(shape=(N, F))
A1_in = Input((N, N))
gc2_1 = conv([X1_in, A1_in])
gc2_1, A1 = mincut([gc2_1,A1_in])
gc2_1 = conv2([gc2_1, A1])
pool_1 =  pool(gc2_1)
d1 = Dense(200,activation='relu')(pool_1)

# Second Graph Layers
X2_in = Input(shape=(N, F))
A2_in = Input((N, N))
gc2_2 = conv([X2_in, A2_in]) # Notice that both graphs shares layers (shared weights)
gc2_2, A2 = mincut([gc2_2,A2_in])
gc2_2 = conv2([gc2_2, A2])
pool_2 = pool(gc2_2)
d2 = Dense(200,activation='relu')(pool_2)

# Dense final layers
merged = Concatenate()([d1, d2])

merged1 = Dense(800,activation='relu')(merged)
merged2 = Dense(32,activation='relu')(merged1)

classe = Dense(n_classes, name="class",activation="softmax")(merged2)
speedup = Dense(1, name="speddup")(merged2)

# Build model
model = Model(inputs=[X1_in, A1_in,X2_in, A2_in], outputs=[classe,speedup])
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss=['categorical_crossentropy', 'mse'],loss_weights=[1, 0.00005], weighted_metrics=['acc'])
model.summary()

In [ ]:
# Train model
#history = model.fit([x_train[:,0,:,:], A_train[:,0,:,:], x_train[:,1,:,:], A_train[:,1,:,:]], [y_train[:,0:4], y_train[:,4]],
#          batch_size=batch_size, validation_split=0.05, epochs=epochs)
history = model.fit(train_generator, validation_data=validation_generator, epochs=epochs)

plt.plot(np.array(history.history['class_acc']))
plt.title('model accuracy')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['treino', 'val'], loc='upper left')
plt.savefig(f"figures/GNN_train.cfg.epochs_{epochs}.batch_{batch_size}.pdf")

In [ ]:
# Evaluate model
print('Evaluating model.')
#eval_results = model.evaluate([x_test[:,0,:,:], A_test[:,0,:,:], x_test[:,1,:,:], A_test[:,1,:,:]], [y_test[:,0:4],y_test[:,4]],
#                              batch_size=batch_size)
eval_results = model.evaluate(test_generator)
print('Done. Test loss: {:.4f}. Test acc: {:.2f}'.format(*eval_results))

In [ ]:
print("Confusion matrix:")
# pred_vals = model.predict([x_test[:,0,:,:], A_test[:,0,:,:], x_test[:,1,:,:], A_test[:,1,:,:]])
pred_vals = model.predict(test_generator)
pred_vals = np.argmax(pred_vals[0], axis=-1)

test_values = []
for i in range(0, len(test_generator)):
    _, (classe, _) = test_generator[i]
    for j in range(0, test_generator.batch_size):
        test_values.append(classe[j])
test_values = np.stack(test_values)    
test_values = np.argmax(test_values, axis=-1)

cm=confusion_matrix(pred_vals,test_values)
print(cm)

labels = ['High Slowdown', 'Low Slowdown', 'Not Significant', 'High Speedup']
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)
ax = sn.heatmap(cm, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='.2f', xticklabels=labels, yticklabels=labels)
for label in ax.get_xticklabels():
    label.set_ha("right")
    label.set_rotation(45)

ax.set_xlabel('Truth Category')
ax.set_ylabel('Predicted Category')

Optionally, we save the trained network for further reuse (model configuration and weights)

In [ ]:
# Saving weights
output_model_file = f'./model_data/ccpe_cfg_GNN.trained.epochs_{epochs}.batch_{batch_size}.h5'
model.save(output_model_file, overwrite=True)
print(f'Model saved to {output_model_file}')